
**Multi-GPU Basics.**

Basic Multi-GPU computation example using TensorFlow library.


In [0]:
import numpy as np
import tensorflow as tf
import datetime

In [0]:
#processing unit logs
log_device_placement=True

#num of multiplications to perform
n=10

In [0]:
#example : compute A^n+B^n on 2 GPUs

#create random large matrix
A = np.random.rand(int(1e4), int(1e4)).astype('float32')
B = np.random.rand(int(1e4), int(1e4)).astype('float32')

#create graph to store results
c1=[]
c2=[]

#define matrix power
def matpow(M,n):
  if n<1: #abstract cases where n<1
    return M
  else:
    return tf.matmul(M,matpow(M,n-1))

In [0]:
-# single GPU computing
with tf.device('/gpu:0'):
  a=tf.constant(A)
  b=tf.constant(B)
  
  #compute A^n+B^n and store it in c1
  c1.append(matpow(a,n))
  c1.append(matpow(b,n))
  
with tf.device('/cpu:0'):
  sum=tf.add_n(c1) #addition of all elements in c1, i.e. A^n+B^n
  
t1_1=datetime.datetime.now()

with tf.Session(config=tf.ConfigProto(log_device_placement=log_device_placement)) as sess:
  #runs the op
  sess.run(sum)

t2_1=datetime.datetime.now()               
                

In [0]:
#multiple gpu computing
# gpu:0 computes A^n
with tf.device('/gpu:0'):
  #compute A^n and store result in c2
  a=tf.constant(A)
  c2.append(matpow(a,n))
  
#gpu:1 computes B^n
with tf.device('/gpu:1'):
  #compute B^n and store result in c2
  b=tf.constant(B)
  c2.append(matpow(b,n))
  
with tf.device('/cpu:0'):
  sum=tf.add_n(c2) # addition of all elements in c2, i.e. A^n+B^n
  
t1_2=datetime.datetime.now()

with tf.Session(config=tf.ConfigProto(log_device_placement=log_device_placement)) as sess:
  # runs the op
  sess.run(sum)
  
t2_2=datetime.datetime.now()  


In [0]:
print("Single GPU computation time "+str(t2_1-t1_1))
print("Multiple GPU computation time "+str(t2_2-t1_2))